In [1]:
import os
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd
from nltk.lm import Vocabulary
import sys
sys.path.append("../../lib")
from metrics import levenshtein
import pickle

In [2]:
folder = "../../data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_training_18M_without_Finnish/NL/"

In [3]:
output_folder = Path("../../data/nl")

In [4]:
files = sorted(os.listdir(folder))
len(files)

1

In [5]:
import glob

files = glob.glob(folder + '/**/*.txt', recursive=True)
len(files)

149

In [6]:
from multiprocessing import Pool

def extract(name):
    with open(name) as file:
        return file.readlines()
    
def create_windows(x):
    A, B, window_length = x
    assert len(A) == len(B)
    return [(A[i:i + window_length], B[i:i + window_length]) 
            for i in range(len(A) + 1)]
    
p = Pool(4)
    
data = list(p.imap_unordered(extract, tqdm(files), chunksize = 128))
len(data)

  0%|          | 0/149 [00:00<?, ?it/s]

149

In [7]:
# data = []
# for f in tqdm(files):
#     with open(f) as file:
#         data.append(file.readlines())

data = pd.DataFrame(data, 
                    columns = ["ocr_to_input", 
                               "ocr_aligned", 
                               "gs_aligned"])\
.assign(ocr_to_input = lambda df: df.ocr_to_input.str.replace("[OCR_toInput] ", "", regex = False),
        ocr_aligned = lambda df: df.ocr_aligned.str.replace("[OCR_aligned] ", "", regex = False),
        gs_aligned = lambda df: df.gs_aligned.str.replace("[ GS_aligned] ", "", regex = False))

print(data.shape)
data.head()

(149, 3)


,ocr_to_input,ocr_aligned,gs_aligned
0,99 16de VERGADERING. — 12 JANUARIJ 1815. Rappo...,@99 @1@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@...,99 (1) De aanspraak van den heer Elout komt o...
1,PAARDEH Katipioonachappon üvpqqy Ruiteraport V...,@PAARDE@@@@@@@@@@@@@@@@@H Ka@@@@@@@@@@@@@@@@@@...,PAARDEN Tweede afdeeling Manchester United-Co...
2,(N*. UIL) 439 XIII. Zamenstélling der regterli...,(N*. UIL) 439 XIII. Zamenstélling der regterli...,@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@...
3,32 7dk VERGADERING. - 23 NOVEMBER 1814. Bereke...,32 7dk VERGADERING. - 23 NOVEMBER 1814. Bereke...,@@@@@@@@@@@@@@@@@@@@@ @3@@@@@@@@@@@@@@@@@@@@@@...
4,LYDÊNS-STOFFEN. $;i knecht des Hoogenpriesters...,LYDÊNS-STOFFEN. $;i knecht des Hoogenpriesters...,@@@@@@@@@@@@@@@@@@@ knecht des Hoogenpriesters...


In [8]:
data.applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,149.000000,149.000000,149.000000
mean,4275.597315,4616.543624,4616.543624
std,3328.913843,3646.668097,3646.668097
min,42.000000,42.000000,42.000000
25%,1573.000000,1593.000000,1593.000000
50%,2963.000000,3129.000000,3129.000000
75%,6765.000000,7164.000000,7164.000000
max,14708.000000,16028.000000,16028.000000


In [9]:
levenshtein(reference = data.gs_aligned.str.replace("@", ""), 
            hypothesis = data.ocr_to_input).cer.describe()

count     149.000000
mean      174.060130
std       783.880180
min         0.361141
25%         6.142668
50%        28.063660
75%        47.859025
max      5250.000000
Name: cer, dtype: float64

In [10]:
levenshtein(reference = data.gs_aligned, 
            hypothesis = data.ocr_aligned).cer.describe()

count    149.000000
mean      28.376282
std       25.365501
min        0.361141
25%        6.090373
50%       26.360519
75%       43.360191
max       99.290780
Name: cer, dtype: float64

In [11]:
vocabulary = Vocabulary(data.ocr_to_input.sum() + data.ocr_aligned.sum() + data.gs_aligned.sum())
print(len(vocabulary))
with open(output_folder/"data/vocabulary.pkl", "wb") as file:
    pickle.dump(vocabulary, file)

170


In [12]:
dev = data.sample(n = 5, random_state = 1)
dev.to_pickle(output_folder/"data/dev.pkl")
dev.shape

(5, 3)

In [13]:
train = data.drop(dev.index)
train.to_pickle(output_folder/"data/train.pkl")
train.shape

(144, 3)

In [14]:
dev.applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,5.000000,5.000000,5.000000
mean,4343.000000,4902.000000,4902.000000
std,2796.406176,3156.806456,3156.806456
min,1653.000000,1669.000000,1669.000000
25%,2711.000000,3061.000000,3061.000000
50%,2824.000000,3666.000000,3666.000000
75%,6260.000000,6543.000000,6543.000000
max,8267.000000,9571.000000,9571.000000


In [15]:
levenshtein(reference = dev.gs_aligned.str.replace("@", ""), 
            hypothesis = dev.ocr_to_input).cer.describe()

count     5.000000
mean     41.297091
std      26.052941
min       3.850782
25%      31.143991
50%      39.770524
75%      65.268346
max      66.451811
Name: cer, dtype: float64

In [16]:
levenshtein(reference = dev.gs_aligned, 
            hypothesis = dev.ocr_to_input).cer.describe()

count     5.000000
mean     38.188531
std      23.955090
min       3.834632
25%      28.193401
50%      37.597432
75%      59.056192
max      62.260997
Name: cer, dtype: float64

In [17]:
window_length = 100

In [18]:
df = train#.head(100)
train_aligned = list(p.imap_unordered(create_windows, 
                                      tqdm(zip(df.ocr_aligned, 
                                               df.gs_aligned, 
                                               [window_length for x in df.ocr_aligned]), 
                                           total = len(df.ocr_aligned)),
                                      chunksize = 128))
s = []
for r in tqdm(train_aligned):
    s.extend(r)
train_aligned = pd.DataFrame(s, columns = ["source", "target"])
print(train_aligned.shape)
train_aligned.head()

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

(663499, 2)


,source,target
0,"C U ) \ maaltyd.kort gemaakt te hebben , naar ...","@@@@@@@ maaltyd kort gemaakt te hebben@, naar ..."
1,"U ) \ maaltyd.kort gemaakt te hebben , naar d...","@@@@@@ maaltyd kort gemaakt te hebben@, naar d..."
2,"U ) \ maaltyd.kort gemaakt te hebben , naar de...","@@@@@ maaltyd kort gemaakt te hebben@, naar de..."
3,") \ maaltyd.kort gemaakt te hebben , naar de ...","@@@@ maaltyd kort gemaakt te hebben@, naar de ..."
4,") \ maaltyd.kort gemaakt te hebben , naar de h...","@@@ maaltyd kort gemaakt te hebben@, naar de h..."


In [19]:
train_aligned = train_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
train_aligned.head()

,source,target
0,"C U ) \ maaltyd.kort gemaakt te hebben , naar ...","@@@@@@@ maaltyd kort gemaakt te hebben@, naar ..."
1,"U ) \ maaltyd.kort gemaakt te hebben , naar d...","@@@@@@ maaltyd kort gemaakt te hebben@, naar d..."
2,"U ) \ maaltyd.kort gemaakt te hebben , naar de...","@@@@@ maaltyd kort gemaakt te hebben@, naar de..."
3,") \ maaltyd.kort gemaakt te hebben , naar de ...","@@@@ maaltyd kort gemaakt te hebben@, naar de ..."
4,") \ maaltyd.kort gemaakt te hebben , naar de h...","@@@ maaltyd kort gemaakt te hebben@, naar de h..."


In [20]:
dev_aligned = dev.apply(lambda r: create_windows((r["ocr_aligned"], r["gs_aligned"], window_length)), 
                            axis = 1).sum()
dev_aligned = pd.DataFrame(dev_aligned, columns = ["source", "target"])
print(dev_aligned.shape)
dev_aligned.head()

(24515, 2)


,source,target
0,t VERANTWOORDING. 7I Iaatfde woorden gaf ik du...,@ VERANTWOORDING. 7I laatste woorden gaf ik du...
1,VERANTWOORDING. 7I Iaatfde woorden gaf ik dui...,VERANTWOORDING. 7I laatste woorden gaf ik dui...
2,VERANTWOORDING. 7I Iaatfde woorden gaf ik duid...,VERANTWOORDING. 7I laatste woorden gaf ik duid...
3,ERANTWOORDING. 7I Iaatfde woorden gaf ik duide...,ERANTWOORDING. 7I laatste woorden gaf ik duide...
4,RANTWOORDING. 7I Iaatfde woorden gaf ik duidel...,RANTWOORDING. 7I laatste woorden gaf ik duidel...


In [21]:
dev_aligned = dev_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
dev_aligned.head()

,source,target
0,t VERANTWOORDING. 7I Iaatfde woorden gaf ik du...,@ VERANTWOORDING. 7I laatste woorden gaf ik du...
1,VERANTWOORDING. 7I Iaatfde woorden gaf ik dui...,VERANTWOORDING. 7I laatste woorden gaf ik dui...
2,VERANTWOORDING. 7I Iaatfde woorden gaf ik duid...,VERANTWOORDING. 7I laatste woorden gaf ik duid...
3,ERANTWOORDING. 7I Iaatfde woorden gaf ik duide...,ERANTWOORDING. 7I laatste woorden gaf ik duide...
4,RANTWOORDING. 7I Iaatfde woorden gaf ik duidel...,RANTWOORDING. 7I laatste woorden gaf ik duidel...


In [22]:
train_aligned.to_pickle(output_folder/"data/train_aligned.pkl")
dev_aligned.to_pickle(output_folder/"data/dev_aligned.pkl")